In [ ]:
!pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete.CPD import TabularCPD

در این جا به کمک مدل شبکه بیز درخت یاد شده را با احتمالات آن میسازیم.

In [ ]:
# Defining the Bayesian Model Structure
model = BayesianModel([('Weather', 'Road'), ('Road', 'Accident')])

cpd_weather = TabularCPD(variable='Weather', variable_card=2, values=[[0.7], [0.3]])
cpd_road = TabularCPD(variable='Road', variable_card=2,
                  values=[[0.9, 0.6], [0.1, 0.4]],
                  evidence=['Weather'], evidence_card=[2])
cpd_accident = TabularCPD(variable='Accident', variable_card=2,
                  values=[[0.95, 0.25], [0.05, 0.75]],
                  evidence=['Road'], evidence_card=[2])

# Associating the CPDs with the network structure
model.add_cpds(cpd_weather, cpd_road, cpd_accident)


/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


از اینجا به بعد به کمک درخت بیزین و در انتها به کمک مارکف تسک‌های خواسته شده را انجام می‌دهیم .

In [ ]:
# Task 1: Calculate the probability of a car accident when the weather is sunny and the road
# is dry.


from pgmpy.inference import VariableElimination

# Create an inference object
infer = VariableElimination(model)

# Compute P(Accident | Road=dry, Weather=sunny)
q = infer.query(['Accident'], evidence={'Road':0 , 'Weather': 0})
print(q)

+-------------+-----------------+
| Accident    |   phi(Accident) |
+=============+=================+
| Accident(0) |          0.9500 |
+-------------+-----------------+
| Accident(1) |          0.0500 |
+-------------+-----------------+


/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [ ]:
# Task 2: Calculate the probability of a car accident when the weather is rainy and the road
# is wet.

# Compute P(Accident | Road=wet, Weather=rainy)
q = infer.query(['Accident'], evidence={'Road':1 , 'Weather': 1})
print(q)

+-------------+-----------------+
| Accident    |   phi(Accident) |
+=============+=================+
| Accident(0) |          0.2500 |
+-------------+-----------------+
| Accident(1) |          0.7500 |
+-------------+-----------------+


In [ ]:
# Task 3: Prove the following conditional probabilities using the chain rule of probability:
# P(Accident=yes|Weather=sunny, Road=dry) = 0.0315
# P(Accident=yes|Weather=rainy, Road=wet) = 0.135

q = infer.query(['Accident'], evidence={'Road':1 , 'Weather': 0})
print(q)



+-------------+-----------------+
| Accident    |   phi(Accident) |
+=============+=================+
| Accident(0) |          0.2500 |
+-------------+-----------------+
| Accident(1) |          0.7500 |
+-------------+-----------------+


In [ ]:
q.values

array([0.25, 0.75])

In [ ]:
q1 = infer.query(['Road'], evidence={'Weather': 0})
print(q1)

+---------+-------------+
| Road    |   phi(Road) |
+=========+=============+
| Road(0) |      0.9000 |
+---------+-------------+
| Road(1) |      0.1000 |
+---------+-------------+


In [ ]:
# Task 4: Calculate the joint probability distribution of all three variables.
import numpy as np

# Get all variables in the model
variables = ['Weather','Road','Accident']
from pgmpy.factors.discrete import JointProbabilityDistribution

weather_0 = 0.7
weather_1 = 0.3
list_res = []
#Weather(0) , Road(0) , Accident(0)
accident_0 = infer.query(['Accident'], evidence={'Road':0}).values[0]
road_0 = infer.query(['Road'], evidence={'Weather': 0}).values[0]
res_0 = accident_0 * road_0 * weather_0
list_res.append(res_0)

#Weather(0) , Road(0) , Accident(1)
accident_1 = infer.query(['Accident'], evidence={'Road':0}).values[1]
road_0 = infer.query(['Road'], evidence={'Weather': 0}).values[0]
res_1 = accident_1 * road_0 * weather_0
list_res.append(res_1)

#Weather(0) , Road(1) , Accident(0)
accident_0 = infer.query(['Accident'], evidence={'Road':1 }).values[0]
road_1 = infer.query(['Road'], evidence={'Weather': 0}).values[1]
res_2 = accident_0 * road_1 * weather_0
list_res.append(res_2)


#Weather(0) , Road(1) , Accident(1)
accident_1 = infer.query(['Accident'], evidence={'Road':1}).values[1]
road_1 = infer.query(['Road'], evidence={'Weather': 0}).values[1]
res_3 = accident_1 * road_1 * weather_1
list_res.append(res_3)




#Weather(1) , Road(0) , Accident(0)
accident_0 = infer.query(['Accident'], evidence={'Road':0}).values[0]
road_0 = infer.query(['Road'], evidence={'Weather': 1}).values[0]
res_4 = accident_0 * road_0 * weather_1
list_res.append(res_4)

#Weather(1) , Road(0) , Accident(1)
accident_1 = infer.query(['Accident'], evidence={'Road':0 }).values[1]
road_0 = infer.query(['Road'], evidence={'Weather': 1}).values[0]
res_5 = accident_1 * road_0 * weather_1
list_res.append(res_5)

#Weather(1) , Road(1) , Accident(0)
accident_0 = infer.query(['Accident'], evidence={'Road':1 }).values[0]
road_1 = infer.query(['Road'], evidence={'Weather': 1}).values[1]
res_6 = accident_0 * road_1 * weather_1
list_res.append(res_6)


#Weather(1) , Road(1) , Accident(1)
accident_1 = infer.query(['Accident'], evidence={'Road':1}).values[1]
road_1 = infer.query(['Road'], evidence={'Weather': 1}).values[1]
res_7 = accident_1 * road_1 * weather_1
list_res.append(res_7)




print(list_res)

# create the Joint Probability Distribution object
joint_prob = JointProbabilityDistribution(variables, [2, 2, 2],list_res/np.sum(list_res))

# print the distribution
print(joint_prob)

# [0.5985, 0.03150000000000001, 0.017499999999999998, 0.0225, 0.17099999999999999, 0.008999999999999998, 0.03, 0.09000000000000001]


[0.5985, 0.031500000000000014, 0.017499999999999998, 0.0225, 0.171, 0.009000000000000003, 0.03, 0.09000000000000001]
+------------+---------+-------------+----------------------------+
| Weather    | Road    | Accident    |   P(Weather,Road,Accident) |
+============+=========+=============+============================+
| Weather(0) | Road(0) | Accident(0) |                     0.6170 |
+------------+---------+-------------+----------------------------+
| Weather(0) | Road(0) | Accident(1) |                     0.0325 |
+------------+---------+-------------+----------------------------+
| Weather(0) | Road(1) | Accident(0) |                     0.0180 |
+------------+---------+-------------+----------------------------+
| Weather(0) | Road(1) | Accident(1) |                     0.0232 |
+------------+---------+-------------+----------------------------+
| Weather(1) | Road(0) | Accident(0) |                     0.1763 |
+------------+---------+-------------+----------------------------+

In [ ]:
import copy
#Task 5: Calculate the marginal probability distribution of each variable.

# Marginal probability distribution of Weather variable
prob_1 = copy.copy(joint_prob)
prob_1.marginal_distribution(['Weather'])
print(prob_1)


+------------+--------------+
| Weather    |   P(Weather) |
+============+==============+
| Weather(0) |       0.6907 |
+------------+--------------+
| Weather(1) |       0.3093 |
+------------+--------------+


In [ ]:

# Marginal probability distribution of Road variable
prob_2 = copy.copy(joint_prob)
prob_2.marginal_distribution(['Road'])
print(prob_2)

+---------+-----------+
| Road    |   P(Road) |
+=========+===========+
| Road(0) |    0.8351 |
+---------+-----------+
| Road(1) |    0.1649 |
+---------+-----------+


In [ ]:

# Marginal probability distribution of Accident variable
prob_3 = copy.copy(joint_prob)
prob_3.marginal_distribution(['Accident'])
print(prob_3)

+-------------+---------------+
| Accident    |   P(Accident) |
+=============+===============+
| Accident(0) |        0.8423 |
+-------------+---------------+
| Accident(1) |        0.1577 |
+-------------+---------------+


In [ ]:
# Task 6: Calculate the conditional probability distribution of Accident given Weather=rainy.

q = infer.query(['Accident'], evidence={'Weather': 1})
print(q)


+-------------+-----------------+
| Accident    |   phi(Accident) |
+=============+=================+
| Accident(0) |          0.6700 |
+-------------+-----------------+
| Accident(1) |          0.3300 |
+-------------+-----------------+


In [ ]:
# Task 7: Using the Markov Blanket of Accident, calculate the conditional probability
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import MarkovNetwork

# define the variables
weather = ['sunny', 'rainy']
road = ['dry', 'wet']
accident = ['no', 'yes']

# create an empty model
model = MarkovNetwork()

# add the variables to the model
model.add_nodes_from(['Weather', 'Road', 'Accident'])

# add factors for the marginal distributions
cpd_weather = TabularCPD(variable='Weather', variable_card=2, values=[[0.7], [0.3]])
cpd_road = TabularCPD(variable='Road', variable_card=2, values=[[0.5], [0.5]])
cpd_accident = TabularCPD(variable='Accident', variable_card=2, values=[[0.5], [0.5]])

# add factors for the conditional probabilities
cpd_road_weather = TabularCPD(variable='Road', variable_card=2, values=[[0.9, 0.1], [0.6, 0.4]], evidence=['Weather'], evidence_card=[2])
cpd_accident_road = TabularCPD(variable='Accident', variable_card=2, values=[[0.95, 0.05], [0.25, 0.75]], evidence=['Road'], evidence_card=[2])

# add the factors to the model
model.add_factors(cpd_weather, cpd_road, cpd_accident, cpd_road_weather, cpd_accident_road)

# add the edges between the variables
model.add_edges_from([('Weather', 'Road'), ('Road', 'Accident')])

# check the model for validity
model.check_model()



True

In [ ]:
print(model)

MarkovNetwork with 3 nodes and 2 edges


In [ ]:
# distribution of Accident given Weather=sunny and Road=wet.

from pgmpy.inference import VariableElimination

# Create the inference engine
infer = VariableElimination(model)

# Compute the CPD
q = infer.query(['Accident'], evidence={'Weather': 0, 'Road': 1})

# Print the result
print(q)


+-------------+-----------------+
| Accident    |   phi(Accident) |
+=============+=================+
| Accident(0) |          0.0053 |
+-------------+-----------------+
| Accident(1) |          0.0788 |
+-------------+-----------------+


In [ ]:
# Task 8: Using the network structure, explain why knowing the value of Weather makes
# Road and Accident conditionally independent











در این مدل، متغیر جاده فرزند متغیر آب و هوا است و والد متغیر تصادف است .بدان مغنی است که وضعیت متغیر آب و هوا بر وضعیت متغیر جاده تاثیر می‌گذارد که به نوبه خود بر وضعیت متغیر تصادف تاثیر می‌گذارد . اگر مقدار آب‌وهوا را بدانیم، اطلاعاتی در مورد وضعیت متغیر مسیر ارائه می‌دهد . به عنوان مثال، اگر آب‌وهوا بارانی است، احتمال مرطوب بودن جاده بیشتر از زمانی است که هوا آفتابی باشد. این بدان معنا است که اگر متغیر آب و هوا را از قبل بدانیم ، یافتن وضعیت متغیر مسیر اطلاغات کمتری در مورد وضعیت متغیر تصادف ارائه می‌دهد . بنابراین،دانستن وضعیت آب‌وهوا، جاده و تصادف را به صورت شرطی مستقل می‌کند .
به عبارت دیگر، تاثیر آب‌وهوا بر وضعیت جاده، اطلاغات کافی را در مورد وضعیت جاده فراهم می‌کند و تاثیر تصادف در جاده را در زمینه تعیین وضعیت جاده با توجه به وضعیت آب‌وهوا، اضافه می‌کند .بنابرین، دو متغیر باتوجه به آب‌وهوا به صورت شرطی مستقل می‌شوند. 


